In [1]:
import sqlite3
import pandas as pd 

conn = sqlite3.Connection('../backend/data.db')
cur = conn.cursor()

cur.execute("""
    select max(article_id)
    from articles
    limit 1
""")

MAX_ARTICLE_ID = cur.fetchone()[0]

cur.execute("""
    select max(chunk_id)
    from chunks
    limit 1
""")

MAX_CHUNK_ID = cur.fetchone()[0]

In [2]:
import pandas as pd 
import json

df = []

with open ('data/scraped.jsonl', 'r') as file:
    for line in file:
        df.append(json.loads(line))

with open ('data/scraped2.jsonl', 'r') as file:
    for line in file:
        df.append(json.loads(line))

df = pd.DataFrame(df)

df = df.sort_values('count', ascending=False)
df = df.drop_duplicates(subset=['url'], keep='first')

df = df.reset_index()
df['article_id'] = df.index + MAX_ARTICLE_ID + 1

df['title'] = df.lower_title
df['clean_title'] = df.lower_title

df = df.drop(['index', 'lower_title'], axis=1)

In [3]:
df.head()

,count,url,text,article_id,title,clean_title
0,49436,https://en.wikipedia.org/wiki/Race_%28United_S...,\nRace and ethnicity in the United States cens...,75197907,race (united states census),race (united states census)
1,38734,https://en.wikipedia.org/wiki/White_%28U.S._Ce...,\nRace and ethnicity in the United States cens...,75197908,white (u.s. census),white (u.s. census)
2,35586,https://en.wikipedia.org/wiki/Marriage,\nMarriage - Wikipedia\nJump to content\nMain ...,75197909,marriage,marriage
3,34251,https://en.wikipedia.org/wiki/Poverty_Line,\nPoverty threshold - Wikipedia\nJump to conte...,75197910,poverty line,poverty line
4,34102,https://en.wikipedia.org/wiki/Census,\nCensus - Wikipedia\nJump to content\nMain me...,75197911,census,census


In [4]:
df.drop('text', axis=1).to_csv('data/scrape_articles_top5_len40_stride15.csv', index=False)

In [10]:
def truncate_text(text):
    if 'References' in text:
        return text.split('References', 1)[0]
    elif 'Notes' in text:
        return text.split('Notes', 1)[0]
    # elif 'Citations' in text:
    #     return text.split('Citations', 1)[0]
    else:
        None

df['mod_text'] = df.text.apply(truncate_text)

In [12]:
print(df.sample().iloc[0]['text'])


Vertebrate - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
		Navigation
	
Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us
		Contribute
	
HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate Create account Log in
		Pages for logged out editors learn more
ContributionsTalk
Contents
move to sidebar
hide
(Top)
1
Characteristics
Toggle Characteristics subsection
1.1
Unique features
1.2
Physical
1.3
Molecular
2
Evolutionary history
Toggle Evolutionary history subsection
2.1
Cambrian explosion: first vertebrates
2.2
Paleozoic: from fish to amphibians
2.3
Mesozoic: from reptiles to mammals and birds
2.4
Cenozoic: Age of Mammals
3
Approaches to classification
Toggle Approaches to classification subsection
3.1
Taxonomic history
3.2
Traditional taxonomy
3.3
External phylogeny
3.4
Internal phylogeny
4
Diversity
Toggle Diversity subsection
4.1
Species by group


In [7]:
MIN_COUNT = 5
STRIDE = 15
LENGTH = 40

In [12]:
import json

buffer = [] 
BUFFER_SIZE = 100000
chunk_id = 0
articles_processed = 0

def clear_buffer(buffer):
    with open(f'data/scrape_chunks_top{MIN_COUNT}_len{LENGTH}_stride{STRIDE}.jsonl', 'a', encoding='utf-8') as file:
        for row in buffer:
            file.write(json.dumps(row) + '\n')
    return []

chunk_id = MAX_CHUNK_ID + 1

for _, row in df.iterrows():
    if articles_processed % 10000 == 0:
        print('processed article:', articles_processed)
    start = 0
    end = LENGTH
    text = row['text'].split()
    while end < len(text):
        buffer.append({
            'article_id': row['article_id'], 
            'chunk_id': chunk_id, 
            'chunk': ' '.join(text[start : end])
        })
        chunk_id += 1
        
        start += STRIDE 
        end += STRIDE

        if len(buffer) >= BUFFER_SIZE:
            buffer = clear_buffer(buffer)
    
    if start <= len(text):
        buffer.append({
            'article_id': row['article_id'], 
            'chunk_id': chunk_id, 
            'chunk': ' '.join(text[start :])
        })
        chunk_id += 1

    articles_processed += 1

if buffer:
    buffer = clear_buffer(buffer)

processed article: 0
